In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold

In [2]:
def transform_data(X):
    """
    This function transforms the 5 input features of matrix X (x_i denoting the i-th component of X) 
    into 21 new features phi(X) in the following manner:
    5 linear features: phi_1(X) = x_1, phi_2(X) = x_2, phi_3(X) = x_3, phi_4(X) = x_4, phi_5(X) = x_5
    5 quadratic features: phi_6(X) = x_1^2, phi_7(X) = x_2^2, phi_8(X) = x_3^2, phi_9(X) = x_4^2, phi_10(X) = x_5^2
    5 exponential features: phi_11(X) = exp(x_1), phi_12(X) = exp(x_2), phi_13(X) = exp(x_3), phi_14(X) = exp(x_4), phi_15(X) = exp(x_5)
    5 cosine features: phi_16(X) = cos(x_1), phi_17(X) = cos(x_2), phi_18(X) = cos(x_3), phi_19(X) = cos(x_4), phi_20(X) = cos(x_5)
    1 constant features: phi_21(X)=1

    Parameters
    ----------
    X: matrix of floats, dim = (700,5), inputs with 5 features

    Returns
    ----------
    X_transformed: array of floats: dim = (700,21), transformed input with 21 features
    """
    X_transformed = np.zeros((len(X), 21))
    
    phi_1 = X[:,0]
    phi_2 = X[:,1]
    phi_3 = X[:,2]
    phi_4 = X[:,3]
    phi_5 = X[:,4]
    phi_6 = phi_1**2
    phi_7 = phi_2**2
    phi_8 = phi_3**2
    phi_9 = phi_4**2
    phi_10 = phi_5**2
    phi_11 = np.exp(phi_1)
    phi_12 = np.exp(phi_2)
    phi_13 = np.exp(phi_3)
    phi_14 = np.exp(phi_4)
    phi_15 = np.exp(phi_5)
    phi_16 = np.cos(phi_1)
    phi_17 = np.cos(phi_2)
    phi_18 = np.cos(phi_3)
    phi_19 = np.cos(phi_4)
    phi_20 = np.cos(phi_5)
    phi_21 = np.ones(len(X))
    X_transformed = np.array([phi_1, phi_2, phi_3, phi_4, phi_5, phi_6, phi_7, phi_8, phi_9, phi_10, phi_11, phi_12, phi_13, phi_14, phi_15, phi_16, phi_17, phi_18, phi_19, phi_20, phi_21]).T

    assert X_transformed.shape == (len(X), 21)
    return X_transformed


In [3]:
def scale_X(X):
    X_scaled = np.ones((len(X), 21))
    X_max = np.max(X, axis=0)
    X_min = np.min(X, axis=0)
    for i in range(20):
        # X_scaled[:,i] = (X[:,i] - X_min[i])/(X_max[i] - X_min[i])
        X_scaled[:,i] = X[:,i]/X_max[i]
    assert X_scaled.shape == (len(X), 21)
    return X_scaled, X_max

In [4]:
def rescale_w(w, X_max, y_max):
    w_rescaled = np.copy(w)
    for i in range(20):
        w_rescaled[i] = w[i] / X_max[i] * y_max
    return w_rescaled

In [5]:
def calculate_RMSE(w, X, y):
    """This function takes test data points (X and y), and computes the empirical RMSE of 
    predicting y from X using a linear model with weights w. 

    Parameters
    ----------
    w: array of floats: dim = (13,), optimal parameters of ridge regression 
    X: matrix of floats, dim = (15,13), inputs with 13 features
    y: array of floats, dim = (15,), input labels

    Returns
    ----------
    RMSE: float: dim = 1, RMSE value
    """
    X_transformed = transform_data(X)
    RMSE = np.sqrt(np.mean((y - np.dot(X_transformed, w))**2))
    assert np.isscalar(RMSE)
    return RMSE


In [6]:
def fit(X, y, lam):
    """
    This function receives training data points, transform them, and then fits the linear regression on this 
    transformed data. Finally, it outputs the weights of the fitted linear regression. 

    Parameters
    ----------
    X: matrix of floats, dim = (700,5), inputs with 5 features
    y: array of floats, dim = (700,), input labels)

    Returns
    ----------
    w: array of floats: dim = (21,), optimal parameters of linear regression
    """
    w = np.zeros((21,))
    X_transformed = transform_data(X)
    X_scaled, X_max = scale_X(X_transformed)
    y_scaled = y / np.max(y)
    w = np.linalg.inv(X_scaled.T @ X_scaled + lam * np.identity(21)) @ X_scaled.T @ y_scaled
    w = rescale_w(w, X_max, np.max(y))
    assert w.shape == (21,)
    return w

In [7]:
def find_lam(X, y):
    lam_list = [0, 0.00003, 0.0001, 0.0003, 0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 1, 3, 10, 30, 100, 300, 1000, 3000, 10000, 30000, 100000]
    RMSE_mat = np.zeros((10, len(lam_list)))
    kf = KFold(n_splits=10, shuffle=True, random_state=2023)
    for i, (train_index, test_index) in enumerate(kf.split(X)):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        for j, lam in enumerate(lam_list):
            w = fit(X_train, y_train, lam)
            RMSE_mat[i, j] = calculate_RMSE(w, X_test, y_test)

    avg_RMSE = np.mean(RMSE_mat, axis=0)

    return lam_list[np.argmin(avg_RMSE)]


In [8]:
# Data loading
data = pd.read_csv("train.csv")
y = data["y"].to_numpy()
data = data.drop(columns=["Id", "y"])
# print a few data samples
print(data.head())

X = data.to_numpy()
# The function retrieving optimal LR parameters
lam = find_lam(X, y)
print('best lam is ', lam)
w = fit(X, y, lam)
# Save results in the required format
RMSE = calculate_RMSE(w, X, y)
print('RMSE is ', RMSE)
np.savetxt("./results.csv", w, fmt="%.12f")

     x1    x2    x3    x4    x5
0  0.02  0.05 -0.09 -0.43 -0.08
1 -0.13  0.11 -0.08 -0.29 -0.03
2  0.08  0.06 -0.07 -0.41 -0.03
3  0.02 -0.12  0.01 -0.43 -0.02
4 -0.14 -0.12 -0.08 -0.02 -0.08
best lam is  0.01
RMSE is  1.9415062480051746
